In [1]:
import time
import lhapdf
import numpy as np
from random import sample
import sys
import matplotlib.pyplot as plt
%matplotlib inline

from pycompressor.pdf_grid import XGrid, PdfSet
from pycompressor.estimators import Estimators
from pycompressor.err_function import compute_erfm
from pycompressor.err_function import compute_erfs
from pycompressor.err_function import compute_erfc
from pycompressor.err_function import ErfComputation
# from pycompressor.compressor import error_function

In [2]:
# Physical Parameters
pdfname = "NNPDF31_nnlo_as_0118_1000"
Qscale  = 1
nf = 2
# construct grid
x = XGrid().build_xgrid()
# x = np.linspace(1e-2,0.9,5)

# Construct Toy Prior set
pdf = PdfSet(pdfname, x, Qscale, nf).build_pdf()
print(f'Initial pdf shape: {pdf.shape}')

def sampling(replica, number):
    """
    Sample a subset a replicas
    """
    # Construct toy Reduced set
    index = np.random.choice(
        replica.shape[0],
        number,
        replace=False
    )
    return replica[index]

# Construc Toy Prior and Reduced sets
prior = sampling(pdf, 100)
print(f'Prior pdf shape: {prior.shape}')
reduc = sampling(prior, 50)
print(f'Reduced pdf shape: {reduc.shape}')

Initial pdf shape: (1000, 5, 104)
Prior pdf shape: (100, 5, 104)
Reduced pdf shape: (50, 5, 104)


In [ ]:
plt.figure()
for p in range(reduc.shape[0]):
    plt.plot(x, reduc[p][3], color="red", alpha=.25)
plt.show()

In [3]:
est_dic = {
    'moment_estimators': ['mean', 'stdev', 'skewness', 'kurtosis'],
    'stat_estimators': ['kolmogorov_smirnov']
}

%time erf = ErfComputation(prior, est_dic, 50)

CPU times: user 17 s, sys: 3.57 ms, total: 17 s
Wall time: 17 s


In [4]:
erf

Normalizations: {'mean': 2491.2089226870703, 'stdev': 11.829688545180119, 'skewness': 17471965.645850815, 'kurtosis': 152097.04169963673, 'kolmogorov_smirnov': 1428.4062879726036}

In [6]:
%time erf.compute_tot_erf(reduc)

CPU times: user 48 ms, sys: 81 µs, total: 48.1 ms
Wall time: 47.2 ms


0.6962637044052908